<a href="https://colab.research.google.com/github/Rohin107/codingground/blob/master/JKM_LSTM_Dense_18122023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import all the required libraries
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM

df = pd.read_csv('/content/JKM_15122023.csv')
df = df.set_index('Date')
df.index = pd.to_datetime(df.index, format='%d-%m-%Y')
df.sort_index(inplace=True)

dataset = df.values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

train_size = int(len(dataset) * 0.90)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

from keras.preprocessing.sequence import TimeseriesGenerator # Generates batches for sequence data
seq_size = length = 45
batch_size = 1
train_generator = TimeseriesGenerator(train,train,length=length,batch_size=batch_size)
# print("Total number of samples in the original training data = ", len(train))
# print("Total number of samples in the generated data = ", len(train_generator))

validation_generator = TimeseriesGenerator(test, test, length=length ,batch_size=batch_size)

num_features = 1
model = Sequential()
model.add(LSTM(90, activation='relu', return_sequences=True, input_shape=(length, num_features)))
model.add(LSTM(45, activation='relu'))
model.add(Dense(45))
model.add(Dense(3))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error', metrics='mean_absolute_error')

model.summary()

model.fit(generator=train_generator, verbose=2, epochs=100, validation_data=validation_generator)

trainPredict = model.predict(train_generator)
testPredict = model.predict(validation_generator)

trainPredict = scaler.inverse_transform(trainPredict)
trainY_inverse = scaler.inverse_transform(train)
testPredict = scaler.inverse_transform(testPredict)
testY_inverse = scaler.inverse_transform(test)

trainScore = math.sqrt(mean_squared_error(trainY_inverse[length:], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(testY_inverse[length:], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[length:len(trainPredict)+length, :] = trainPredict

testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(seq_size*2)-1:len(dataset)-1, :] = testPredict

plt.plot(dataset)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()